In [1]:
import pickle
import copy
import numpy as np
import random
import os
import torch
os.chdir('..')
from test_utils import *
from ipywidgets import interact


# 데이터 직접 입력

In [2]:
import argparse
os.chdir('..')
from lib.utils.learning import *
from lib.utils.tools import *

In [3]:
from test_utils import *

h36m_connections = [
    (0,1),
    (0,4),
    (0,7),
    (1,2),
    (2,3),
    (4,5),
    (5,6),
    (7,8),
    (8,9),
    (9,10),
    (8,14),
    (14,15),
    (15,16),
    (8,11),
    (11,12),
    (12,13)
]

In [4]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--config", type=str, default="/home/hrai/codes/MotionBERT/configs/pose3d/MB_ft_h36m.yaml", help="Path to the config file.")
    parser.add_argument('-c', '--checkpoint', default='', type=str, metavar='PATH', help='checkpoint directory')
    parser.add_argument('-p', '--pretrained', default='', type=str, metavar='PATH', help='pretrained checkpoint directory')
    parser.add_argument('-r', '--resume', default='', type=str, metavar='FILENAME', help='checkpoint to resume (file name)')
    parser.add_argument('-e', '--evaluate', default='/home/hrai/codes/MotionBERT/checkpoint/pose3d/MB_ft_h36m/best_epoch.bin', type=str, metavar='FILENAME', help='checkpoint to evaluate (file name)')
    #parser.add_argument('-ms', '--selection', default='', type=str, metavar='FILENAME', help='checkpoint to finetune (file name)')
    parser.add_argument('-sd', '--seed', default=0, type=int, help='random seed')
    opts = parser.parse_args([])
    return opts

def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
os.chdir('..')
opts = parse_args()
set_random_seed(opts.seed)
args = get_config(opts.config)

In [5]:
model_backbone = load_backbone(args)
if torch.cuda.is_available():
    model_backbone = nn.DataParallel(model_backbone)
    model_backbone = model_backbone.cuda()

print('Loading checkpoint', opts.evaluate)
checkpoint = torch.load(opts.evaluate, map_location=lambda storage, loc: storage)
model_backbone.load_state_dict(checkpoint['model_pos'], strict=True)
model_pos = model_backbone
model_pos.eval()

Loading checkpoint /home/hrai/codes/MotionBERT/checkpoint/pose3d/MB_ft_h36m/best_epoch.bin


DataParallel(
  (module): DSTformer(
    (joints_embed): Linear(in_features=3, out_features=512, bias=True)
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks_st): ModuleList(
      (0): Block(
        (norm1_s): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        (norm1_t): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        (attn_s): Attention(
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=512, out_features=512, bias=True)
          (qkv): Linear(in_features=512, out_features=1536, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (attn_t): Attention(
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=512, out_features=512, bias=True)
          (qkv): Linear(in_features=512, out_features=1536, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2_s): LayerNorm((512,), eps=1e-06,

In [6]:
# 예제 CLIP 파일
os.chdir('/home/hrai/codes/MotionBERT')
# with open('data/motion3d/MB3D_f243s81/AIHUB_243/train/00000025.pkl', 'rb') as f:
# 	test = pickle.load(f)
# frame_num = 150

# with open('data/motion3d/MB3D_f243s81/AIHUB_SPORTS_243/train/00000050.pkl', 'rb') as f:
# 	test = pickle.load(f)
# frame_num = 100

with open('data/motion3d/MB3D_f243s81/AIHUB_30_243/train/00000007.pkl', 'rb') as f:
	test = pickle.load(f)
frame_num = 200
batch_input = torch.tensor(test['data_input'][frame_num].reshape(1, 1, 17, 3))

In [7]:
result = model_pos(batch_input.float())
result_array = result[0][0].cpu().detach().numpy()

In [8]:
result_array.shape

(17, 3)

In [9]:
# 2d input
aihub_2d = copy.deepcopy(test['data_input'][frame_num])
# 2d skeleton
pos = copy.deepcopy(aihub_2d)
points_aihub_2d = {}
for i, p in enumerate(pos):
    p = np.append(p, 0.0)
    #print(i, h36m_keypoints[i], p)
    points_aihub_2d[i] = p
pos_vector = pos.reshape(-1) # (51,)
total_min, total_max = pos_vector.min(), pos_vector.max()
print("total min: ", total_min,
      "total max: ", total_max)

interact(draw_skeleton_2d(points_aihub_2d, h36m_connections))

total min:  -0.39021474 total max:  1.0098552


In [10]:
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.gridspec as gridspec

plt.switch_backend('TkAgg')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [11]:
# gt
aihub_3d = copy.deepcopy(test['data_label'][frame_num])
aihub_3d.shape

(17, 3)

In [12]:
aihub_3d[1,1]

0.048183262

In [13]:
visualize_3d_pose([get_rootrel_pose(aihub_3d*1000*3), result_array*1000*3])

In [14]:
MPJPE_for_single_pose(get_rootrel_pose(aihub_3d*1000*3), result_array*1000*3)

189.21756

### ------------ Previous visualization ---------------

In [7]:
# gt
aihub_3d = copy.deepcopy(test['data_label'][frame_num])
pos = copy.deepcopy(aihub_3d)
points_aihub_3d = {}
for i, p in enumerate(pos):
    #print(i, h36m_keypoints[i], p)
    points_aihub_3d[i] = np.array([p[0], -p[2], -p[1]]) #np.array([p[0], p[1], p[2]])  #np.array([p[0], -p[2], p[1]])
pos_vector = pos.reshape(-1) # (51,)
total_min, total_max = pos_vector.min(), pos_vector.max()
print("total min: ", total_min,
      "total max: ", total_max)

fig = draw_skeleton(points_aihub_3d, h36m_connections)
fig.show()

NameError: name 'test' is not defined

In [24]:
# result
aihub_3d = copy.deepcopy(result_array)
pos = copy.deepcopy(aihub_3d)
points_aihub_3d = {}
for i, p in enumerate(pos):
    #print(i, h36m_keypoints[i], p)
    points_aihub_3d[i] = np.array([p[0], -p[2], -p[1]]) #np.array([p[0], p[1], p[2]])  #np.array([p[0], -p[2], p[1]])
pos_vector = pos.reshape(-1) # (51,)
total_min, total_max = pos_vector.min(), pos_vector.max()
print("total min: ", total_min,
      "total max: ", total_max)

fig = draw_skeleton(points_aihub_3d, h36m_connections)
fig.show()

total min:  -0.27778554 total max:  0.24165273


### --------------------------------------------------------------------

### from alphapose result

In [15]:
ap_result = readJSON("/home/hrai/codes/AlphaPose/examples/squat_test_front/alphapose-results.json")
len(ap_result)

268

In [16]:
joint_2d = np.array(ap_result[0]['keypoints']).reshape(-1, 26, 3)

# halpe to h36m
joint_2d = halpe2h36m(joint_2d)

joint_2d.shape

(1, 17, 3)

In [28]:
img = get_2d_pose_image(joint_2d[0], np.ones((1080,1920,3), dtype=np.uint8)*255)
plot_cv2_image(img)

In [18]:
# normalize
res_w, res_h = 1920, 1080
test = copy.deepcopy(joint_2d)
test = test / res_w * 2 - [1, res_h / res_w, 0]


In [19]:
test

array([[[-0.04957975,  0.02099031,  0.00103653],
        [-0.10540543,  0.02099031,  0.00094048],
        [-0.12401396,  0.22568455,  0.00096388],
        [-0.15192687,  0.41177018,  0.00098317],
        [ 0.00624593,  0.03029461,  0.00094887],
        [ 0.03415877,  0.22568455,  0.00100762],
        [ 0.05276737,  0.41177018,  0.00099876],
        [-0.04957975, -0.14648676,  0.00102309],
        [-0.04957975, -0.31396383,  0.00100965],
        [-0.04957975, -0.38839808,  0.00100389],
        [-0.04957975, -0.47213661,  0.00096928],
        [ 0.04346307, -0.26744242,  0.00097386],
        [ 0.08998451, -0.12787819,  0.00099246],
        [ 0.05276737, -0.13718246,  0.00095532],
        [-0.14262257, -0.26744242,  0.0009677 ],
        [-0.18914394, -0.12787819,  0.0009931 ],
        [-0.1612311 , -0.12787819,  0.00101646]]])

In [20]:
batch_input = torch.tensor(test.reshape(1, 1, 17, 3)).float()

In [21]:
from lib.utils.utils_data import flip_data
flip = True
with torch.no_grad():
    if torch.cuda.is_available():
        batch_input = batch_input.cuda()
        if flip:    
            batch_input_flip = flip_data(batch_input)
            predicted_3d_pos_1 = model_pos(batch_input)
            predicted_3d_pos_flip = model_pos(batch_input_flip)
            predicted_3d_pos_2 = flip_data(predicted_3d_pos_flip)                   # Flip back
            predicted_3d_pos = (predicted_3d_pos_1+predicted_3d_pos_2) / 2
        else:
            predicted_3d_pos = model_pos(batch_input)

In [22]:
predicted_3d_pos.shape

torch.Size([1, 1, 17, 3])

In [23]:
result_array = predicted_3d_pos[0][0].cpu().detach().numpy()
#result_array[:, 2] = 0
r_hip = result_array[1]
r_knee = result_array[2]
r_ankle = result_array[3]
l_hip = result_array[4]
l_knee = result_array[5]
l_ankle = result_array[6]


In [24]:
r_ankle, l_ankle

(array([-0.10127847,  0.40188408,  0.15028226], dtype=float32),
 array([0.10055399, 0.40259418, 0.16433024], dtype=float32))

In [25]:
np.linalg.norm(r_hip - r_knee), np.linalg.norm(l_hip - l_knee)

(0.23471133, 0.22845855)

In [26]:
r_ankle[2] = 0
l_ankle[2] = 0

In [29]:
visualize_3d_pose([result_array*1000])

In [30]:
result_array[:, :2]*1000 + [960, 540]

array([[ 960.01708001,  539.88380296],
       [ 887.62931061,  533.91296482],
       [ 884.29216003,  756.10748291],
       [ 858.72153473,  941.88409424],
       [1032.29949188,  541.53457046],
       [1044.31208801,  758.56784058],
       [1060.5539856 ,  942.59417725],
       [ 960.97166884,  379.30859375],
       [ 960.34158   ,  216.49423218],
       [ 961.1679064 ,  146.47982788],
       [ 960.41588023,   65.0244751 ],
       [1055.72901154,  259.76501465],
       [1103.65150452,  413.82083893],
       [1063.40990448,  407.62533569],
       [ 866.4430542 ,  257.37307739],
       [ 818.52003479,  407.54321289],
       [ 850.11457825,  412.80162048]])

In [39]:
img = get_2d_pose_image(result_array[:, :2]*1000  + [1920.0/2, 1080.0/2], np.ones((1080,1920,3), dtype=np.uint8)*255)
plot_cv2_image(img)

In [ ]:
projected_2d = result_array[:, :2]*1000  + [1920.0/2, 1080.0/2]


In [32]:
### 여러 프레임
joint_2ds = []
for i in range(len(ap_result)):
    joint_2ds.append(np.array(ap_result[i]['keypoints']).reshape(26, 3))
#np.array(joint_2ds).shape
joint_2ds = halpe2h36m(np.array(joint_2ds))

In [33]:
joint_2ds.shape

(268, 17, 3)

In [34]:
# normalize
joint_2ds = joint_2ds / res_w * 2 - [1, res_h / res_w, 0]

In [43]:
# 하나를 여러개
batch_input = torch.tensor(joint_2ds[100].reshape(-1, 1, 17, 3)).repeat(1, 243, 1, 1)
batch_input.shape

torch.Size([1, 243, 17, 3])

In [44]:
result = model_pos(batch_input.float())
print(result.shape)

torch.Size([1, 243, 17, 3])


In [45]:
result_array = result[0][0].cpu().detach().numpy()
visualize_3d_pose([result_array*1000])

In [42]:
joint_2ds.shape

(268, 17, 3)

In [40]:
# 여러 프레임
n = 243
batch_input = torch.tensor(joint_2ds[:n].reshape(-1, n, 17, 3))
batch_input.shape

torch.Size([1, 243, 17, 3])

In [41]:
result = model_pos(batch_input.float())
print(result.shape)
result_array = result[0][0].cpu().detach().numpy()
visualize_3d_pose([result_array*1000])

torch.Size([1, 243, 17, 3])
